In [ ]:
# Empty the output folder
#!rm -rf /kaggle/working/*

In [ ]:
import pandas as pd
from datasets import Dataset
# import tensorflow as tf
import numpy as np
import json
import re

## 1) Preprocess Data

In [ ]:
# Load the dataset as a json file
file_json = open("/kaggle/input/medicalmeadow-multiplechocie/medical_meadow_medqa.json")
data = json.load(file_json)

# Now let's try with the whole dataset
#data = data[:200]

In [ ]:
def is_json_empty(json_obj):
    # return true if length is 0.
    return len(json_obj) == 0

In [ ]:
# The possible choices are all under the 'input' key
# We need to separate them and return the result
# As a dictionary
def process_record(record):

    input_text = record['input']
    output_text = record['output']

    answer_str_index = input_text.find('{')
    input_question = input_text[:answer_str_index].strip()

    answer_options_str = input_text[answer_str_index:].strip()

    answer_options_str = answer_options_str.replace("'", "\"")

    if answer_options_str.endswith(','):
                answer_options_str = answer_options_str[:-1]

    # Parse the multiple choices string into a dictionary
    answer_options = json.loads(answer_options_str)

    # Extract the correct answer
    correct_answer = output_text.split(':')[0].strip()

    # Create a dictionary for the DataFrame
    processed_record = {
        'input': input_question,
        'A': answer_options.get('A', ''),
        'B': answer_options.get('B', ''),
        'C': answer_options.get('C', ''),
        'D': answer_options.get('D', ''),
        'E': answer_options.get('E', ''),
        'correct_answer': correct_answer
    }
    return processed_record

processed_data = []

# Process all records
# We have set a try/except to avoid interruptin the process when we encounter a badly formatted record
# Unfortunately for this experiment we are going to just discard those
# Sorry but I didn't find a fast solution :(

for record in data:
    if not is_json_empty(record):  # Ensure record is not empty
        try:
            processed_data.append(process_record(record))
        except:
            print('error')

# Create a DataFrame
df = pd.DataFrame(processed_data)

# Display the DataFrame
print(df.head())

# Display the DataFrame size
print(len(df.index))

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

In [ ]:
# Some parameters
model_checkpoint = "google-bert/bert-base-cased"
batch_size = 2

In [ ]:
# Set up the list of options
options = ['A', 'B', 'C', 'D', 'E']

# Define a mapping of string values to integer values
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
reverse_mapping = {0: 'A', 1:'B', 2:'C', 3:'D', 4:'E'}

In [ ]:
# We apply the mapping to the correct answers
df['correct_answer'] = df['correct_answer'].map(mapping)

In [ ]:
# Convert the Pandas dataframe into Dataset
datasets = Dataset.from_pandas(df)

# We rename the correct_answer column into label (will be useful later)
datasets = datasets.rename_column("correct_answer", "label")

# Print informations
datasets

# Retrieve the first record
datasets[0]

{'input': 'Q:A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. She otherwise feels well and is followed by a doctor for her pregnancy. Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air. Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. Which of the following is the best treatment for this patient??',
 'A': 'Ampicillin',
 'B': 'Ceftriaxone',
 'C': 'Ciprofloxacin',
 'D': 'Doxycycline',
 'E': 'Nitrofurantoin',
 'label': 4}

In [ ]:
# To load a pre-trained model of Bert and download the vocabulary
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    # Repeat each prompt for 5 times to go with the 5 possibilities of each option
    first_sentences = [[context] * 5 for context in examples["input"]]
    # Grab all options

    second_sentences = [examples[options[i]] for i in range(5)]

    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, padding = False, truncation=True)
    # Un-flatten
    return {
        k: [v[i : i + 5] for i in range(0, len(v), 5)]
        for k, v in tokenized_examples.items()
    }

In [ ]:
# Now, it is the time to tokenize the complete Dataset
encoded_datasets = datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/9415 [00:00<?, ? examples/s]

## 2) Fine tuning Bert

In this experiment we are going to use Keras and not Pytorch, since a previous version that used Pytorch couldn't start training because of insufficient memory. This problems didn't arise using Keras.

In [ ]:
from transformers import TrainingArguments, Trainer, TFAutoModelForMultipleChoice

model = TFAutoModelForMultipleChoice.from_pretrained(model_checkpoint)

2024-05-21 19:49:35.354137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 19:49:35.354240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 19:49:35.651248: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForMultipleChoice.

Some weights or buffers of the TF 2.0 model TFBertForMultipleChoice were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Model parameters
learning_rate = 3e-4
batch_size = batch_size
num_train_epochs = 10
#num_train_epochs = 1
report_to=None
token=False
push_to_hub=False

In [ ]:
# To pad the input datasets in order to transform the input datasets with the same size

from dataclasses import dataclass
from transformers.tokenization_utils_base import (
    PreTrainedTokenizerBase,
    PaddingStrategy,
)
from typing import Optional, Union
import tensorflow as tf


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)]
            for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="np",
        )

        # Un-flatten
        batch = {
            k: tf.reshape(v, (batch_size, num_choices, -1)) for k, v in batch.items()
        }
        # Add back labels
        batch["labels"] = tf.convert_to_tensor(labels, dtype=tf.int64)
        return batch

In [ ]:
data_collator = DataCollatorForMultipleChoice(tokenizer)

# Split the data into train, test and validation sets
# 80% training, 14% validation, 6% test
encoded_datasets_train_valtest = encoded_datasets.train_test_split(test_size=0.2, seed=42)
encoded_datasets_val_test = encoded_datasets_train_valtest['test'].train_test_split(test_size=0.3, seed=42)

# Prepare train, test and validation datasets using prepare_tf_dataset
train_set = model.prepare_tf_dataset(
    encoded_datasets_train_valtest['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

val_set = model.prepare_tf_dataset(
    encoded_datasets_val_test['train'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

test_set = model.prepare_tf_dataset(
    encoded_datasets_val_test['test'],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

all_set = model.prepare_tf_dataset(
    encoded_datasets,
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
# Define the optimizer
from transformers import create_optimizer

total_train_steps = (len(train_set) // batch_size) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps
)

In [ ]:
# Compile the model

model.compile(
    optimizer=optimizer,
    metrics=["accuracy"],
)

In [ ]:
# Print model's summary

model.summary()

Model: "tf_bert_for_multiple_choice"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
 dropout_37 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  769       
                                                                 
Total params: 108311041 (413.17 MB)
Trainable params: 108311041 (413.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Set Keras callback for EarlyStopping
# We are going to monitor the validation loss
from tensorflow import keras

callback = keras.callbacks.EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
# Set mixed precision
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
# Evaluate the vanilla model on the test set

model.evaluate(
    test_set,
    batch_size=batch_size,
    verbose='auto',
    callbacks=[callback],
    return_dict=False,
)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
282/282 [==============================] - 104s 175ms/step - loss: 1.6136 - accuracy: 0.2074


[1.6136046648025513, 0.20744681358337402]

In [ ]:
# Train the model

model.fit(
    train_set,
    validation_data=val_set,
    epochs=num_train_epochs,
    callbacks=[callback],
)

Epoch 1/10


I0000 00:00:1716321165.788571      86 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3766/3766 [==============================] - 2229s 570ms/step - loss: 1.6222 - accuracy: 0.1942 - val_loss: 1.6094 - val_accuracy: 0.1935
Epoch 2/10
3766/3766 [==============================] - 2126s 564ms/step - loss: 1.6173 - accuracy: 0.2001 - val_loss: 1.6094 - val_accuracy: 0.2215
Epoch 3/10
3766/3766 [==============================] - 2121s 563ms/step - loss: 1.6181 - accuracy: 0.1961 - val_loss: 1.6094 - val_accuracy: 0.2071
Epoch 4/10
3766/3766 [==============================] - 2126s 564ms/step - loss: 1.6201 - accuracy: 0.1916 - val_loss: 1.6094 - val_accuracy: 0.1783
Epoch 5/10
3766/3766 [==============================] - 2123s 564ms/step - loss: 1.6159 - accuracy: 0.1997 - val_loss: 1.6094 - val_accuracy: 0.2170


In [ ]:
# Save the model
model.save('/kaggle/working/model.keras')

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


## 3) Predict the test set

In [ ]:
# The following function gets the indices of the highest scoring answers for each row
# and converts them back to our answer format (A, B, C, D, E)

import numpy as np
def predictions_to_map_output(predictions):
    sorted_answer_indices = np.argsort(-predictions)
    top_answer_indices = sorted_answer_indices[:,:3] # Get the first three answers in each row
    top_answers = np.vectorize(reverse_mapping.get)(top_answer_indices)
    return np.apply_along_axis(lambda row: ' '.join(row), 1, top_answers)

In [ ]:
# Test Data
# use encoded_dataset_val_test['test']

first_sentence = [encoded_datasets_val_test['test'][0]['input'] for i in range(5)]

second_sentence = [encoded_datasets_val_test['test'][0]['A'], encoded_datasets_val_test['test'][0]['B'], encoded_datasets_val_test['test'][0]['C'],
                   encoded_datasets_val_test['test'][0]['D'], encoded_datasets_val_test['test'][0]['E']]

In [ ]:
# Gives back the letter from the int
# TODO find a better way, this is horrible

def from_int_to_label(answer, test_set):
    if(answer == 0):
        return test_set['A']
    elif(answer == 1):
        return test_set['B']
    elif(answer == 2):
        return test_set['C']
    elif(answer == 3):
        return test_set['D']
    else:
        return test_set['E']

In [ ]:
# Let's adapt it to Bert

first = first_sentence
second = second_sentence

tokenized = tokenizer(first, second, padding="longest", return_tensors="np")
tokenized_1 = {key: np.expand_dims(array, 0) for key, array in tokenized.items()}

# Best Prediction
outputs = model(tokenized_1).logits

# print(outputs.shape)
answer = np.argmax(outputs)

print('======= Test Case 1 =======')
print(f"Question: {first[0]}\n")
print(f"The answer is choice {answer}: {second[answer]}")
print(f"The correct answer is: ", from_int_to_label(encoded_datasets_val_test['test'][0]['label'], encoded_datasets_val_test['test'][0]))

======= Test Case 1 =======
Question: Q:You are the intern on the labor and delivery floor. Your resident asks you to check on the patient in Bed 1. She is a 27-year-old prima gravida with no significant past medical history. She has had an uncomplicated pregnancy and has received regular prenatal care. You go to her bedside and glance at the fetal heart rate tracing (Image A). What is the most likely cause of this finding??

The answer is choice 4: Congenital heart block
The correct answer is:  Fetal head compression


In [ ]:
# Evaluate the fine-tuned model on the test set

model.evaluate(
    test_set,
    batch_size=batch_size,
    verbose='auto',
    callbacks=[callback],
    return_dict=False,
)

282/282 [==============================] - 52s 184ms/step - loss: 1.6094 - accuracy: 0.2447


[1.609436273574829, 0.24468085169792175]

## References


*   https://huggingface.co/docs/transformers/tasks/multiple_choice
*   https://www.kaggle.com/code/bennyfung/bert-model-for-multiple-choice/notebook#2.-Dataset

